In [ ]:
import os
import numpy as np
from qiskit.qasm2 import dump
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import efficient_su2
from qiskit.synthesis.qft import synth_qft_full

np.random.seed(42)
path = "./programs"


def Adder(n_qubits: int, use_toffoli: bool = False, save_qasm: bool = False) -> QuantumCircuit:
    if n_qubits % 2 != 0:
        raise ValueError(f"n_qubits must be even, but got {n_qubits}")
    n_bits = n_qubits // 2 - 1
    circuit = QuantumCircuit(n_qubits)

    def _toffoli(x: int, y: int, z: int):
        circuit.h(z)
        circuit.cx(y, z)
        circuit.tdg(z)
        circuit.cx(x, z)
        circuit.t(z)
        circuit.cx(y, z)
        circuit.t(y)
        circuit.tdg(z)
        circuit.cx(x, z)
        circuit.cx(x, y)
        circuit.t(z)
        circuit.h(z)
        circuit.t(x)
        circuit.tdg(y)
        circuit.cx(x, y)

    def _MAJ(x: int, y: int, z: int, use_toffoli: bool):
        circuit.cx(z, y)
        circuit.cx(z, x)
        if use_toffoli:
            circuit.ccx(x, y, z)
        else:
            _toffoli(x, y, z)

    def _UMA(x: int, y: int, z: int, use_toffoli: bool):
        circuit.x(y)
        circuit.cx(x, y)
        if use_toffoli:
            circuit.ccx(x, y, z)
        else:
            _toffoli(x, y, z)
        circuit.x(y)
        circuit.cx(z, x)
        circuit.cx(z, y)

    ind = [2 * i + 2 for i in range(n_bits)]
    for i in ind:
        _MAJ(i - 2, i - 1, i, use_toffoli)
    circuit.cx(ind[-1], n_qubits - 1)
    for i in reversed(ind):
        _UMA(i - 2, i - 1, i, use_toffoli)

    if save_qasm:
        filename = os.path.join(path, f"Adder{n_qubits}.qasm")
        dump(circuit, filename)
    return circuit


def BV(n_qubits: int, save_qasm: bool = False) -> QuantumCircuit:
    circuit = QuantumCircuit(n_qubits)
    circuit.x(n_qubits - 1)
    circuit.h(range(n_qubits))
    for i in range(n_qubits - 1):
        circuit.cx(i, n_qubits - 1)
    circuit.h(range(n_qubits))

    if save_qasm:
        filename = os.path.join(path, f"BV{n_qubits}.qasm")
        dump(circuit, filename)
    return circuit


def GHZ(n_qubits: int, save_qasm: bool = False) -> QuantumCircuit:
    circuit = QuantumCircuit(n_qubits)
    circuit.h(0)
    for i in range(n_qubits - 1):
        circuit.cx(i, i + 1)

    if save_qasm:
        filename = os.path.join(path, f"GHZ{n_qubits}.qasm")
        dump(circuit, filename)
    return circuit


def QAOA(n_qubits: int, save_qasm: bool = False) -> QuantumCircuit:
    circuit = efficient_su2(n_qubits, entanglement="linear", reps=1)
    params = np.random.uniform(-np.pi, np.pi, size=circuit.num_parameters)
    circuit = circuit.assign_parameters(params)
    if save_qasm:
        filename = os.path.join(path, f"QAOA{n_qubits}.qasm")
        dump(circuit, filename)
    return circuit


def QFT(n_qubits: int, do_swap: bool = False, save_qasm: bool = False) -> QuantumCircuit:
    circuit = QuantumCircuit(n_qubits)
    for i in range(n_qubits):
        circuit.h(i)
        for j in range(i + 1, n_qubits):
            angle = np.pi / (2 ** (j - i))
            circuit.cp(angle, j, i)
    circuit = transpile(circuit, basis_gates=["h", "rz", "cx"])

    if do_swap:
        for i in range(n_qubits // 2):
            circuit.swap(i, n_qubits - i - 1)

    if save_qasm:
        filename = os.path.join(path, f"QFT{n_qubits}.qasm")
        dump(circuit, filename)
    return circuit


n_qubits = 128
circuit = Adder(n_qubits, save_qasm=False)
print(circuit.count_ops())
# circuit.draw("mpl", fold=0)

In [ ]:
import os
import re


def convert_sqrt_qasm(input_file: str):
    path = "./programs"
    input_file = os.path.join(path, input_file)
    with open(input_file, "r") as f:
        lines = f.readlines()

    reg_map = {}
    start_idx = 0
    converted_lines = []
    reg_prefix = ["x_", "a", "b", "qt"]

    for line in lines:
        line = line.strip()
        if not line or not line.startswith("qreg"):
            continue
        for prefix in reg_prefix:
            pattern = rf"({prefix}\w*)\[(\d+)\]"
            match = re.search(pattern, line)
            if match:
                reg_name = match.group(1)
                reg_size = int(match.group(2))
                if reg_name not in reg_map:
                    reg_map[reg_name] = (start_idx, reg_size)
                    start_idx += reg_size
    n_qubits = start_idx

    converted_lines = []
    for line in lines:
        line = line.strip()
        if not line or line.startswith("qreg") or line.startswith("creg") or line.startswith("reset") or line.startswith("measure"):
            continue
        if line.startswith("include"):
            line += f"\nqreg q[{n_qubits}];"

        for prefix in reg_prefix:
            pattern = rf"({prefix}\w*)\[(\d+)\]"

            def replace_func(match):
                reg_name = match.group(1)
                reg_idx = int(match.group(2))
                if reg_name in reg_map:
                    base_idx, _ = reg_map[reg_name]
                    return f"q[{base_idx + reg_idx}]"
                return match.group(0)

            line = re.sub(pattern, replace_func, line)

        converted_lines.append(line)

    n_qubits = start_idx
    output_file = f"SQRT{n_qubits}.qasm"
    output_file = os.path.join(path, output_file)
    with open(output_file, "w") as f:
        f.write("\n".join(converted_lines))


for n in [4, 7, 9]:
    convert_sqrt_qasm(f"square_root_n{n}.qasm")